# Finding concentrations of crowded housing

[Emily Alpert Reyes](https://twitter.com/latimesemily), previously on the demographics beat at the L.A. Times, came to the Data Desk one day with a hunch. After grabbing a Census table from American Fact-Finder and manipulating some columns in Excel, she noticed that a few places in Los Angeles had high rates of "crowding" -- that is, had high concentrations of homes in an area that housed more than one person per room.

But just the rate of crowding could be misleading. At the very top of the list were some rural areas that had very few homes, all of which were crowded. How to square that with ZIP codes in Los Angeles County that had thousands of homes, and a 60% rate of crowding?

We decided to rank the country by a one-proportion z-value. For every ZIP code, we calculated the following:

$$ z = \frac{ \hat{ p } - p_0 }{ \sqrt{ \frac{ p_0 (1 - p_0) }{ n } } } $$

Where $ n $ was the number of homes in the ZIP, $ x $ the number of crowded homes and $ \hat{p} = \frac{x}{n} $. 

$ p_0 $ was the nationwide mean rate of crowding. It was a constant (about 3 percent or .03) in every calculation. 

The value it spits out is effectively the number of standard deviations away from the mean each ZIP is. I liked this metric because it would take into account not just the percentage of crowded homes ($\hat{p}$ in the numerator) but also how many homes were in the area ($n$ in the denominator). So if one area had six crowded homes out of 10 and another had 60 out of 100, the second place would have a higher z-value, since as $ n $ gets larger, $\sqrt{n}$ in the denominator gets smaller and the z-value gets bigger.

The story that combined my analysis and Emily's reporting ran as the A1 centerpiece on March 17, 2014: ["L.A. and Orange counties are an epicenter of overcrowded housing"](http://www.latimes.com/local/la-me-crowding-20140308-story.html)

### Getting the data

Below I'll run through how I used R to do the calculations. While Emily actually gave me her Excel sheet to work with, I'll start by pulling the data with the `acs` package in R, which Rob Barry detailed how to use in [this blog post](http://rob-barry.com/2014/08/23/Identifying-the-Divide/).

In [1]:
library(acs)


Attaching package: ‘acs’

The following object is masked from ‘package:base’:

    apply



The table we need is B25014, Tenure by Occupants per Room. If we had to, we could look for all tables with "room" in it like this:

In [2]:
head(unique(acs.lookup(endyear = 2012, span = 5, table.name = "room", case.sensitive = F)@results[, c("table.number", "table.name")]))

,table.number,table.name
1,B25014,Tenure by Occupants per Room
14,B25014A,OCCUPANTS PER ROOM (WHITE ALONE HOUSEHOLDER)
17,B25014B,OCCUPANTS PER ROOM (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER)
20,B25014C,OCCUPANTS PER ROOM (AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER)
23,B25014D,OCCUPANTS PER ROOM (ASIAN ALONE HOUSEHOLDER)
26,B25014E,OCCUPANTS PER ROOM (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE HOUSEHOLDER)


Note: First-time users of the acs package will need to grab a developers key from the Census website and run this line once.

In [3]:
# api.key.install(key='YOUR_KEY')

Next let's grab all of the data in that table.

In [4]:
# geography identifier, set to grab all zip codes
zcta_geo <- geo.make(zip.code = '*')

# pulling the data set
CROWD_DATA <- acs.fetch(endyear = 2012, span = 5, geo = zcta_geo, table.number = 'B25014', col.names = 'pretty')

Examining the structure shows this is everything you would get if you downloaded an Excel file from Fact-Finder, it just takes a little work to figure out where the relevant information is.

In [5]:
str(CROWD_DATA)

Formal class 'acs' [package "acs"] with 9 slots
  ..@ endyear       : int 2012
  ..@ span          : int 5
  ..@ geography     :'data.frame':	33120 obs. of  2 variables:
  .. ..$ NAME                 : chr [1:33120] "ZCTA5 00601" "ZCTA5 00602" "ZCTA5 00603" "ZCTA5 00606" ...
  .. ..$ zipcodetabulationarea: chr [1:33120] "00601" "00602" "00603" "00606" ...
  ..@ acs.colnames  : chr [1:13] "Tenure by Occupants per Room: Total:" "Tenure by Occupants per Room: Owner occupied:" "Tenure by Occupants per Room: Owner occupied: 0.50 or less occupants per room" "Tenure by Occupants per Room: Owner occupied: 0.51 to 1.00 occupants per room" ...
  ..@ modified      : logi FALSE
  ..@ acs.units     : Factor w/ 5 levels "count","dollars",..: 1 1 1 1 1 1 1 1 1 1 ...
  ..@ currency.year : int 2012
  ..@ estimate      : num [1:33120, 1:13] 5477 13241 17518 1754 9181 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:33120] "ZCTA5 00601" "ZCTA5 00602" "ZCTA5 00603" "ZCTA5 00606" ...
  .. ..

Here are the estimates for each ZIP Code Tabulation Area or ZCTA, the Census Bureau's analog for USPS ZIP codes:

In [6]:
head(CROWD_DATA@estimate)

,Tenure by Occupants per Room: Total:,Tenure by Occupants per Room: Owner occupied:,Tenure by Occupants per Room: Owner occupied: 0.50 or less occupants per room,Tenure by Occupants per Room: Owner occupied: 0.51 to 1.00 occupants per room,Tenure by Occupants per Room: Owner occupied: 1.01 to 1.50 occupants per room,Tenure by Occupants per Room: Owner occupied: 1.51 to 2.00 occupants per room,Tenure by Occupants per Room: Owner occupied: 2.01 or more occupants per room,Tenure by Occupants per Room: Renter occupied:,Tenure by Occupants per Room: Renter occupied: 0.50 or less occupants per room,Tenure by Occupants per Room: Renter occupied: 0.51 to 1.00 occupants per room,Tenure by Occupants per Room: Renter occupied: 1.01 to 1.50 occupants per room,Tenure by Occupants per Room: Renter occupied: 1.51 to 2.00 occupants per room,Tenure by Occupants per Room: Renter occupied: 2.01 or more occupants per room
ZCTA5 00601,5477,3525,1899,1506,120,0,0,1952,1148,683,59,31,31
ZCTA5 00602,13241,10372,6091,3968,259,38,16,2869,1284,1481,80,0,24
ZCTA5 00603,17518,11621,7692,3678,226,16,9,5897,2962,2642,199,65,29
ZCTA5 00606,1754,1199,589,562,48,0,0,555,313,163,65,14,0
ZCTA5 00610,9181,7126,3797,2901,368,60,0,2055,958,952,140,5,0
ZCTA5 00612,23268,16677,10391,5971,207,108,0,6591,3375,2923,223,45,25


For the purposes of this calculation, that means we had to write a little code to sum those columns where the occupancy exceeded one person per room (an academic definition for crowding). Then, we could divide by the total column to get our percentages.

You can subset on an acs class just like you would subset on a normal data frame. A quick `grep` function call over column names can get us the three columns that will comprise our numerator:

In [7]:
crowded <- CROWD_DATA[,grep(pattern = '1.01|1.51|2.01', x = CROWD_DATA@acs.colnames)]

In [8]:
crowded_total <- apply(crowded, 2, sum)

The `divide.acs` function let's you calculate a percentage like this:

In [9]:
crowding_pct <- divide.acs(numerator = crowded_total, denominator = CROWD_DATA[,1], method = 'proportion')
head(crowding_pct)

Warning message:
In .acs.divider(num = numerator, den = denominator, proportion = T, : ** due to the nature of some of the errors, using the more conservative formula for RATIOS, which assumes that numerator is not a subset of denominator **

ACS DATA: 
 2008 -- 2012 ;
  Estimates w/90% confidence intervals;
  for different intervals, see confint()
            ( aggregate / Tenure by Occupants per Room: Total: )
ZCTA5 00601 0.0440021909804638 +/- 0.0167324718434627           
ZCTA5 00602 0.0314930896457971 +/- 0.00876405665249096          
ZCTA5 00603 0.0310537732617879 +/- 0.0082865249968786           
ZCTA5 00606 0.072405929304447 +/- 0.0379552461514904            
ZCTA5 00610 0.062411502015031 +/- 0.0171142598109101            
ZCTA5 00612 0.0261303077187554 +/- 0.00657596365814987          

Let's put together all of our relevant data in a processed data.frame:

In [10]:
crowding_usa <- data.frame(
    zcta = geography(crowding_pct)[[1]],
    total = as.numeric(estimate(CROWD_DATA[,1])),
    crowded = as.numeric(estimate(crowded_total)),
    pct = as.numeric(estimate(crowding_pct))
)
head(crowding_usa)

,zcta,total,crowded,pct
1,ZCTA5 00601,5477,241,0.04400219
2,ZCTA5 00602,13241,417,0.03149309
3,ZCTA5 00603,17518,544,0.03105377
4,ZCTA5 00606,1754,127,0.07240593
5,ZCTA5 00610,9181,573,0.0624115
6,ZCTA5 00612,23268,608,0.02613031


A quick plot illustrates the issue we are dealing with here:

In [11]:
png("misc-files/crowding-r-plot.png", width=16, height=8, units="in", res=80)
plot(crowding_usa$total, crowding_usa$pct, xlab = 'Number of homes', ylab = "Percentage of crowded homes")
dev.off()

pdf 
  2

<img src = "misc-files/crowding-r-plot.png">

There are a number of ZIP codes with greater than 60% crowding, but it's obvious they are very small and don't have many homes. During our first discussion as a team, we thought about not considering ZIP codes below a certain threshold for the number of homes. But whatever cutoff we set would inherently be arbitrary, and we would be eliminating parts of the country where real-life humans live.

### Calculating the z-value

So we can capture all of the country, we turn to the z-value.

Here's a glimpse at the data right now, sorted by the percentage of homes that are crowded.

In [12]:
head(crowding_usa[order(crowding_usa$pct, decreasing = TRUE),], n = 20)

,zcta,total,crowded,pct
11130,ZCTA5 34268,48,48,1
13591,ZCTA5 42163,15,15,1
27584,ZCTA5 79258,5,5,1
28316,ZCTA5 81655,13,13,1
30474,ZCTA5 92304,15,15,1
30708,ZCTA5 93220,8,8,1
31519,ZCTA5 95680,22,22,1
27417,ZCTA5 78802,100,97,0.97
12312,ZCTA5 38047,73,60,0.8219178
29303,ZCTA5 85654,62,50,0.8064516


Before we calculate the z-value, let's first let's figure out the nationwide rate of crowding. Simply, what percentage of homes in the country are crowded?

In [13]:
p_0 <- sum(crowding_usa$crowded) / sum(crowding_usa$total)
p_0

[1] 0.03245804

It's about 3 percent. So we'll set that as a constant in the calculation, $p_0$. If we were conducting a hypothesis test (which we're not), $p_0$ is the value we expect the proportion to be. 

The z-value will measure the deviation from 3 percent for every ZIP code in the country. Here's how we can write a column of z-values into our data (compare this to the formula spelled out in the intro):

In [14]:
crowding_usa$zval <- (crowding_usa$pct - p_0) / sqrt(p_0 * (1 - p_0) / crowding_usa$total)

Now let's take a look at the ZIP codes ranked by z-values in decreasing order:

In [15]:
crowding_usa <- crowding_usa[order(crowding_usa$zval, decreasing = TRUE),]

In [16]:
head(crowding_usa, n = 20)

,zcta,total,crowded,pct,zval
30020,ZCTA5 90011,21919,9242,0.4216433,325.1402
30016,ZCTA5 90006,18404,7847,0.4263747,301.5535
30097,ZCTA5 90255,18161,7301,0.4020153,281.0318
30621,ZCTA5 92701,12263,5748,0.4687271,272.6193
2008,ZCTA5 07055,20083,7411,0.3690186,269.142
30059,ZCTA5 90057,15764,6297,0.3994545,260.0153
30079,ZCTA5 90201,24399,7858,0.3220624,255.2672
30622,ZCTA5 92703,12901,4920,0.3813658,223.628
30032,ZCTA5 90023,10814,4271,0.394951,212.7143
30013,ZCTA5 90003,16399,5279,0.3219099,209.1649


These ZIPs at the top are what we termed our "most unusual." Most of the ones that start with 9 are in Los Angeles or Orange counties. The top two, [90011](http://maps.latimes.com/neighborhoods/neighborhood/historic-south-central/?q=Los+Angeles%2C+CA+90011%2C+USA&lat=34.007889&lng=-118.2585096&g=Geocodify) and [90006](http://maps.latimes.com/neighborhoods/neighborhood/pico-union/?q=Los+Angeles%2C+CA+90006%2C+USA&lat=34.0470832&lng=-118.2965121&g=Geocodify), were the adjacent neighborhoods of Historic South-Central and Pico-Union in Central Los Angeles.

The z-value wasn't anything we would run with for publication but the top one basically tells you that 90011 is 325 standard deviations (or standard errors) from the mean of 3 percent. 

That calculation was all we needed for a final data set we would query from for the story (we did the same process on a data set of all census tracts as well).

Emily's lede focused on a home in Historic South-Central, under which some queries from our analysis made for nut grafs:

> Cano and her family live in one of the most crowded neighborhoods in the country. Nearly 45% of the homes there are considered "crowded" — having more than one person per room, excluding bathrooms, according to an analysis of Census Bureau data spanning 2008 to 2012. Almost one home in six is severely crowded, with more than two people per room.

> Southern California is an epicenter for crowded housing: Out of the most heavily crowded 1% of census tracts across the country, more than half are in Los Angeles and Orange counties, a Times statistical analysis found. They are sprinkled throughout areas such as Westlake and Huntington Park around Los Angeles, and Santa Ana and Anaheim in Orange County.

> From the outside looking in, it is a largely invisible phenomenon. Places such as Maywood and Huntington Park, south of Los Angeles, look little like the high-rises of Chicago or Boston. Yet behind the closed doors of small bungalows or squat apartment buildings, they are home to thousands more people per square mile than those large cities.

From there, we also made a [map that focused on the top 1% of ZIP codes](http://graphics.latimes.com/crowding-map/), along with some small multiples of sorts as a guide:

<img src="misc-files/crowding-online-map.png">

The [print story](http://www.latimes.com/local/la-me-crowding-20140308-story.html) also ran with a map of Census tracts, with a focus on L.A. and Orange counties:

<img src="misc-files/crowding-analysis.png">

Some additional links related to this analysis:
* [A short, published explanation](http://www.latimes.com/local/la-me-crowding-box-20140308-story.html) of how we did the analysis, or what we at the Data Desk sometimes refer to as a "nerd box." This is where I tried to explain all of the above in simple language.
* Curbed LA, which tracks all things housing in the area, [linked to our map](http://la.curbed.com/archives/2014/03/historic_southcentral_has_the_most_crowded_housing_in_the_us.php).
* Emily wrote about her reporting process on the demographics beat and this story specifically [for Source](https://source.opennews.org/en-US/learning/finding-stories-census-data/).